In [16]:
#AudioClipExtraction
!pip install PyQt5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
#AudioClipExtraction
from PyQt5.QtCore import *
class AudioClipExtractor(QThread):
     def __init__(self,parent):
        QThread.__init__(self, parent)
        
     def run(self):
        self.audio_events()
     def audio_events(self):  
         import librosa 
         import matplotlib.pyplot as plt 
         import numpy as np
         import pandas as pd
         from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
         import moviepy.editor as mp
         import wave 
         from glob import glob
         import wave
         import scipy.io.wavfile as wav
         import numpy as np 

         file_path =glob.glob('/content/OrignalVideo.mp4')

         filename=mp.VideoFileClip(file_path[0])
         filename.audio.write_audiofile('./OrignalVideo/Highlights' +'.wav')
         filename='./OrignalVideo/Highlights.wav'

         sample_rate, data= wav.read(filename)
         length = data.shape[0] / sample_rate
         print(f"Duration of Whole Video in Seconds = {length}s")
         print(f"Number of channels = {data.shape[1]}")
         

         #Convert to mono channel
         data = data.astype(np.float32)
         def read_data(data):
             yield (data[:,0]+data[:,1]/2)
         for i in  read_data(data):
             wave_array=i

         def find_energy(moving_average_list):
             energy=[]
             for array in moving_average_list:
                 yield np.square(array)
         def subsample(array,step):
             ssarray=array[::step].copy()
             return ssarray
         
         #Calling Methods   
         SAMPLING_FACTOR=4
         sampled_array=subsample(wave_array, SAMPLING_FACTOR)
         energy=list(find_energy(sampled_array)) 
        
        #Find The exact time in secwhere the noise comes 
         def find_highlight_times(array, threshold, sample_rate):
           chunks = []
           for i in range(len(array)-1):
             if (array[i]<threshold<array[i+1] or array[i]>threshold>array[i+1]):
               yield (i*SAMPLING_FACTOR/sample_rate)
        
         threshold=np.max(energy)*(4/5)
         times = list (find_highlight_times(energy, threshold, sample_rate))


        #Removing the same secs from lists
         new_list=[]
         for i in times:
             if int(i) not in new_list:
                 new_list.append(int(i))
        
        
        #Removing also the clips or sec which have duration of 20 sec 
         i = 0
         while i < len(new_list)-1:
            if new_list[i+1] - new_list[i]<=20:
                new_list.pop(i)
            else:
                i += 1   
            
        #Making Clips list        
         Clips_list=[]
         for i in new_list:
             if i>=10 or i<=length-10:
                 Clips_list.append([i-10,i+10])  
        
         #Making Clips List         
         for i in Clips_list:
             start_lim=i[0]
             end_lim=i[1]
             filename= './Runs/Events/'+str(start_lim) + "_" +str(end_lim)  + ".mp4"
             ffmpeg_extract_subclip(file_path[0],start_lim,end_lim,targetname=filename)




In [18]:
import sys

class ProgressBar(object):
    DEFAULT_BAR_LENGTH = 60
    DEFAULT_CHAR_ON  = '>'
    DEFAULT_CHAR_OFF = ' '
    prog = 0

    def __init__(self, end, start=0):
        self.end    = end
        self.start  = start
        self._barLength = self.__class__.DEFAULT_BAR_LENGTH

        self.setLevel(self.start)
        self._plotted = False

    def setLevel(self, level):
        self._level = level
        if level < self.start:  self._level = self.start
        if level > self.end:    self._level = self.end

        self._ratio = float(self._level - self.start) / float(self.end - self.start)
        self._levelChars = int(self._ratio * self._barLength)

    def plotProgress(self):
        global prog
        self.prog = int(self._ratio * 100.0)
        sys.stdout.write("\r  %3i%% [%s%s]" %(
        int(self._ratio * 100.0),
        self.__class__.DEFAULT_CHAR_ON  * int(self._levelChars),
        self.__class__.DEFAULT_CHAR_OFF * int(self._barLength - self._levelChars),
        ))
        sys.stdout.flush()
        self._plotted = True

    def setAndPlot(self, level):
        oldChars = self._levelChars
        self.setLevel(level)
        if (not self._plotted) or (oldChars != self._levelChars):
            self.plotProgress()

    def __add__(self, other):
        assert type(other) in [float, int], "can only add a number"
        self.setAndPlot(self._level + other)
        return self
    def __sub__(self, other):
        return self.__add__(-other)
    def __iadd__(self, other):
        return self.__add__(other)
    def __isub__(self, other):
        return self.__add__(-other)

    def __del__(self):
        sys.stdout.write("\n")

In [19]:
import cv2
from PyQt5.QtCore import *
#from ProgressBar import ProgressBar
class FramesExtractor(QThread):
    change_value = pyqtSignal(int)
    def __init__(self,parent, cap,total,span,TotalVideoLength,FramePath,framespersecond):
        QThread.__init__(self, parent)
        self.init(cap,total,span,TotalVideoLength,FramePath,framespersecond)
    def init (self,cap,total,span,TotalVideoLength,FramePath,framespersecond):
        self.cap = cap
        self.total = total
        self.span = span
        self.TotalVideoLength = TotalVideoLength
        self.FramePath = FramePath
        self.framespersecond = framespersecond
    def run(self):
        self.record_frames_manual(self.cap,self.total,self.span,self.TotalVideoLength,self.FramePath,self.framespersecond)
        
    def record_frames_manual(self,cap,total,span,TotalVideoLength,FramePath,framespersecond):
        x=0
        pb = ProgressBar(self.TotalVideoLength)
        value=[0]
        seconds=1
        while True:
            grabbed, frame= self.cap.read()
            if not grabbed:
                break
            if self.total % self.span ==0:
       			
                if self.framespersecond != value[0]:
       				
                    filename = str(self.FramePath)+"/image" + "_"+str(int(seconds))+"."+str(int(x))+ ".png"
                    x=x+1
                else:
                    seconds=seconds+1
                    x=0
                    filename = str(self.FramePath)+"/image" +"_"+str(int(seconds))+"."+str(int(x)) + ".png"
                    x=x+1
                    value=[0]	
                value[0]=int(value[0])+int(self.span)
                frame=cv2.resize(frame,(1080,720))
                #progress(total,TotalVideoLength)
                pb=pb+self.span
                cv2.imwrite(filename, frame)
                
            self.total += 1
            self.change_value.emit(pb.prog)

In [22]:
#text recognition

import cv2
from PyQt5.QtCore import *
#from ProgressBar import ProgressBar
class TextRecognizer(QThread):
    change_value = pyqtSignal(int)
    Score_list_value=pyqtSignal(list)
    def __init__(self,parent,ref_point,FramePath,all_files):
        QThread.__init__(self, parent)
        self.init(ref_point,FramePath,all_files)
    def init (self,ref_point,FramePath,all_files):
        self.ref_point = ref_point
        self.all_files = all_files
        self.FramePath = FramePath
        

    def run(self):
        self.text_recog(self.ref_point,self.FramePath,self.all_files)
        
    def text_recog(self,ref_point,FramePath,all_files):

        import matplotlib.pyplot as plt
        from PIL import Image 
        import pytesseract  
        import tensorflow as tf
        import cv2
        import numpy as np
        import  PreProcessing as tm
        from ProgressBar import ProgressBar
        import os
        
        Scores_history=[]
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        config = ('-l eng --oem 3 --psm 10') 
        pb = ProgressBar(len(self.all_files))
        for i in self.all_files:
            new_img=cv2.imread(i)
            crop_img=new_img[self.ref_point[0][1]:self.ref_point[1][1], self.ref_point[0][0]:self.ref_point[1][0]] 
            grayImage = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            #There is a need of transformation Law when images brighteness is faint, so we use it for now as comment
            Processed_Image=tm.PowerTransformation(grayImage)
            pb=pb+1
            if pb.prog<100:
                
                self.change_value.emit(pb.prog)
            else:
                self.change_value.emit(99)
                
            text=pytesseract.image_to_string(Processed_Image,config=config)
            Scores_history.append(text)      
            text=""
        self.Score_list_value.emit(Scores_history)
        
        print("\nScores Processes SuscessFully!")

In [23]:
#text detection

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog,QLabel,QWidget,QApplication,QProgressBar,QVBoxLayout,QDialog,QMainWindow
from PyQt5.uic import loadUi
from PyQt5.QtGui import *
from PyQt5.QtCore  import *
#from TextRecognition import TextRecognizer
#from AudioClipExtraction import AudioClipExtractor
import time

class TextDetectionWindow(QMainWindow):
    
    def __init__(self,widget,FramesPath,ref_point,FilePath):
        self.FilePath = FilePath
        self.ref_point = ref_point
        self.file = str('./Frames')
        self.widget = widget
        self.FramesPath = FramesPath
        super(TextDetectionWindow,self).__init__()
        loadUi("./Ui_s/TextDetectionUi.ui",self)
        self.Scores_history=[]
        self.button_connections()
        
    def button_connections(self):
        #Next Button
        self.btn_next.setEnabled(True)
        self.btn_next.clicked.connect(lambda: self.btn_next_ftn())
        
        #Back Button
        self.btn_back.clicked.connect(lambda: self.btn_back_ftn())
        
        #Proceed Button
        self.btn_proceed.clicked.connect(lambda: self.btn_proceed_ftn())
        
        #Home Button
        self.btn_home.clicked.connect(lambda: self.btn_home_ftn())
    
    
    def btn_next_ftn(self):
        from Genetic_Algo_Ui import Genetic_Algo_ui
        gen_algo_win = Genetic_Algo_ui(self.widget,self.FilePath,self.ref_point,self.FramesPath)
        self.widget.addWidget(gen_algo_win)
        self.widget.setFixedHeight(300)
        self.widget.setFixedWidth(500)
        self.widget.setCurrentIndex(self.widget.currentIndex()+1)

  
    def btn_back_ftn(self):
        from SelectImageUi import SelectImageWindow
        select_img_win = SelectImageWindow(self.widget,self.FramesPath,self.FilePath)
        self.widget.addWidget(select_img_win)
        self.widget.setFixedHeight(375)
        self.widget.setFixedWidth(872)
        self.widget.setCurrentIndex(self.widget.currentIndex()+1)
    
    def btn_home_ftn(self):
        import sys
        from Main_ui import Window
        main_window = Window(self.widget, self.FilePath)
        self.widget.addWidget(main_window)
        self.widget.setFixedHeight(500)
        self.widget.setFixedWidth(1000)
        self.widget.setCurrentIndex(self.widget.currentIndex()+1)
        
    def updateProgressBar(self,val):
        self.progBar.setValue(val)
    def Scorelistval(self,val):
        self.Scores_history=val
    def sort_seq(self):
        #Reading All Images in Sequenece Wise


        import glob 
        import natsort as nt
        
        path = self.FramesPath # use your path
        all_files = glob.glob(path + "/*.png")
        self.all_files=nt.natsorted(all_files)
        print("sorted")
    
    def make_dirs(self):
        #Making Directories

        import os
        path = './Runs'
            
        self.SubPath_4=path+'/Fours'
        self.SubPath_6=path+'/Sixes' 
        self.SubPath_Out=path+'/Outs' 
        self.SubPath_Events=path+'/Events' 
        
        try:  
            
            if os.path.exists(path):
                    import shutil
                    shutil.rmtree(path)
            else:
                pass
            
            os.makedirs(path,exist_ok=True)        
            os.makedirs(self.SubPath_4,exist_ok=True)
            os.makedirs(self.SubPath_6,exist_ok=True)
            os.makedirs(self.SubPath_Out,exist_ok=True)
            os.makedirs(self.SubPath_Events,exist_ok=True)
    
        except OSError as error:  
            print("Unable to create a drirectory Directory Already Exixts!")
    def Crop_Video(self,start_time, end_time, targetname):
        orignal_path= self.FilePath
        from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
        ffmpeg_extract_subclip(orignal_path, start_time, end_time, targetname)
    
    def classify_events(self):
        import cv2 
        import os
        Fisrt_iteration=True  
        folder_counter_4=1 
        folder_counter_Out=1
        counting_that_place=0
        
        for i,j in zip(self.Scores_history,range(1,len(self.all_files)+1)):
            if '-' in i:  
                i=i.strip()
                temp=i.split('-')      
                if temp[0].isdigit() and temp[1].isdigit() :
            
                    if Fisrt_iteration == True:
                        Score_prev=int(temp[0])
                        Score_index_prev=j-1
                        Out_prev=int(temp[1])
                        Out_index_prev=j-1
                        Fisrt_iteration=False
                        continue              
                    else:
                        Score_curr=int(temp[0])
                        Out_curr=int(temp[1])
                        
                        if (Score_curr-Score_prev)==4:
                            
                  
                            #Splitting the filname and getting the seconds info from it 
                            second=self.all_files[j].split('_')
                            second=second[1].split('.')
                            seconds=second[0]
                            start_time=int(seconds)-15
                            end_time=int(seconds)+15
                    
                            
                            path4_subfolder=self.SubPath_4+"/"+str(start_time)+str("_")+str(end_time)+".mp4"
                            
                            self.Crop_Video(start_time,end_time,path4_subfolder )
                            
                            try:  
                                os.makedirs(path4_subfolder,exist_ok=True)
                            except OSError as error:  
                                print("Unable to create a folder_4 ")   
                                
                                
                                
                            Score_prev=Score_curr
                            Out_prev=Out_curr
                            Score_index_prev=j-1
                            Out_index_prev=j-1
                         
                        elif (Score_curr-Score_prev)==6:
                            
                            
                        
                            #Splitting the filname and getting the seconds info from it 
                            second=self.all_files[j].split('_')
                            second=second[1].split('.')
                            seconds=second[0]
                            start_time=int(seconds)-15
                            end_time=int(seconds)+15
                            
                            
                            path6_subfolder=self.SubPath_6+"/"+str(start_time)+str("_")+str(end_time)+".mp4"
                            
                            self.Crop_Video(start_time,end_time,path6_subfolder )
                            
                            try:  
                                os.makedirs(path6_subfolder,exist_ok=True)
                            except OSError as error:  
                                print("Unable to create a folder_6 ")   
                                
                            Score_prev=Score_curr
                            Out_prev=Out_curr
                            Score_index_prev=j-1
                            Out_index_prev=j-1   
                            
                        elif (Out_curr-Out_prev)==1:
                            
                            #Splitting the filname and getting the seconds info from it 
                            second=self.all_files[j].split('_')
                            second=second[1].split('.')
                            seconds=second[0]
                            start_time=int(seconds)-15
                            end_time=int(seconds)+15
                            
                        
                            pathOUT_subfolder=self.SubPath_Out+"/"+str(start_time)+str("_")+str(end_time)+".mp4"
                            self.Crop_Video(start_time,end_time,pathOUT_subfolder )
                            try:  
                                os.makedirs(pathOUT_subfolder,exist_ok=True)
                            except OSError as error:  
                                print("Unable to create a folder_OUT ")   
                                
        
                               
                            Score_prev=Score_curr
                            Out_prev=Out_curr
                            Score_index_prev=j-1
                            Out_index_prev=j-1
                            
                            
                        else:
                            Score_prev=Score_curr
                            Out_prev=Out_curr
                            Score_index_prev=j-1
                            Out_index_prev=j-1
    def audio_events(self):
        self.thread1 = AudioClipExtractor(self)
        self.thread1.start()
        self.thread1.finished.connect(self.onComplete_AudioClips)
    def DeleteRedundantEvents(self):
        import glob 
        import natsort as nt
        import os 
        
        Path_Four_files= glob.glob(".\\Runs\\Fours"+"\\*.mp4")
        Path_Six_files= glob.glob(".\\Runs\\Sixes"+"\\*.mp4")
        Path_Out_files= glob.glob(".\\Runs\\Outs"+"\\*.mp4")
        Path_Events_files= glob.glob(".\\Runs\\Events"+"\\*.mp4")
        
        All_Filenames=[]
        
        for i in Path_Four_files:
            All_Filenames.append(i)
        for i in Path_Six_files:
            All_Filenames.append(i)
        for i in Path_Out_files:
            All_Filenames.append(i)
        for i in Path_Events_files:
            All_Filenames.append(i)
        
        Delete_items=[]
        for i in range(len(All_Filenames)):
            curr_file=All_Filenames[i].split('\\')
            curr_file=curr_file[3].split('_')
            curr_file=curr_file[1].split('.')
            curr_file=curr_file[0]
            for j in range(i+1,len(All_Filenames)):
                next_file=All_Filenames[j].split('\\')
                next_file=next_file[3].split('_')
                next_file=next_file[0]
                if int(curr_file)> int(next_file):
                    #The above if else is used is curr_num is greater than next_num then it show be checked in that way
                    if (int(curr_file)-int(next_file))<=10 and All_Filenames[i] not in Delete_items:
                        Delete_items.append(All_Filenames[i])
                        Delete_items.append(All_Filenames[j])
                        print('File i: ',All_Filenames[i],' File j: ',All_Filenames[j])
                        
                else:
                    
                    if (int(next_file)-int(curr_file))<=10 and All_Filenames[i] not in Delete_items:
                        Delete_items.append(All_Filenames[i])
                        Delete_items.append(All_Filenames[j])
                        print('File i: ',All_Filenames[i],' File j: ',All_Filenames[j])
        
        Delete_item=[]    
        for i in Delete_items:
            if i not in Delete_item:
                Delete_item.append(i)
                
        for (i,j) in zip(Delete_item,range(len(Delete_item))):
            if 'Events' in i:
                
                os.remove(i)
                Delete_item.pop(j)
                
        print(Delete_item)
    
        
    def onComplete_AudioClips(self):
        
        self.DeleteRedundantEvents()
        self.prog_label.setText("Wait For Audio Completition Part...")
        
        self.updateProgressBar(100)
        
        msg = QtWidgets.QMessageBox()
        msg.setIcon(QtWidgets.QMessageBox.Information)
        msg.setText("All Clips Extracted SuccessFully")
        msg.setWindowTitle("Information")
        msg.exec_()   
        self.prog_label.setText("Completed Successfully!")  
        
        
    def onComplete(self):
        #Exceute te Remaining Part When Thread Completes
        self.prog_label.setText("Making Directories for Events......")
        self.make_dirs()
        time.sleep(2)
        self.prog_label.setText("Storing Events in the directories......")
        self.classify_events()
        self.audio_events()
                           
              
    def btn_proceed_ftn(self):
        self.prog_label.setText("Sorting extracted frames in sequence........")
        self.sort_seq()
        time.sleep(1)
        self.prog_label.setText("Detecting Scoreboard text & recognizing text......")

        self.thread = TextRecognizer(self,self.ref_point,self.FramesPath,self.all_files)
        self.thread.start()
        self.thread.change_value.connect(self.updateProgressBar)
        self.thread.Score_list_value.connect(self.Scorelistval)
        self.thread.finished.connect(self.onComplete)
        